# Tensor-Flow

## Daniel Wilcox: 19147414

This example problem can be found within chapter 11 of the "Hands-on Machine Learning with Scikit-Learn and TensorFlow" by Aurélien Géron. 


In [1]:
import tensorflow as tf

import time
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [2]:
#MNist Data:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

# Exercises:

## 8.
### 8.1) 
Build a DNN with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.


   

In [3]:
#Hidden Layers and neurons per layer:
n_layers = 5
n_neurons = 100

#He initialization:
he_init = tf.variance_scaling_initializer()

#ELU activation:
elu_act = tf.nn.elu

#Function to make dnn layers of equal size:
def make_eq_dnn(inputs, n_hidden_layers=n_layers, n_neurons=n_neurons, 
        initializer=he_init, activation=elu_act, name=None):
    
    with tf.variable_scope(name, "dnn"):
        
        for layer in range(n_hidden_layers):
            inputs = tf.layers.dense(inputs, n_neurons, activation=activation,
                                     kernel_initializer=initializer,
                                     name="hidden%d" % (layer + 1))
        return inputs


In [4]:
reset_graph()

# MNIST
n_inputs = 28 * 28 # MNIST

#Input layer/labels
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

#hidden layers
dnn_layers = make_eq_dnn(X) 


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


### 8.2)
Using Adam optimization and early stopping, try training it on MNIST but only on digits 0 to 4, as we will use transfer learning for digits 5 to 9 in the next exercise. You will need a softmax output layer with five neurons, and as always make sure to save checkpoints at regular intervals and save the final model so you can reuse it later.

In [5]:
n_outputs = 5 #Same as n_layers
    
#Output layer:


logits = tf.layers.dense(dnn_layers, n_outputs, kernel_initializer=he_init, name="logits")
Y_proba = tf.nn.softmax(logits, name="Y_proba")

In [6]:
#Cost function:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
#Adam Optimization:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss, name="training_op")

#Evaluation:    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [7]:
#Dataset for values 0-4
X_train_low = X_train[y_train < 5]
y_train_low = y_train[y_train < 5]

X_valid_low = X_valid[y_valid < 5]
y_valid_low = y_valid[y_valid < 5]

X_test_low = X_test[y_test < 5]
y_test_low = y_test[y_test < 5]

In [8]:
n_epochs = 2000
batch_size = 50

Early_stop = 15
Early_checker = 0
best_loss = np.infty

#Model Training:
with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        
        rand_idx = np.random.permutation(len(X_train_low))
        
        for rand_idx in np.array_split(rand_idx, len(X_train_low)//batch_size):
            X_batch, y_batch = X_train_low[rand_idx], y_train_low[rand_idx]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            
        val_loss, val_acc = sess.run([loss, accuracy], feed_dict={X: X_valid_low, y: y_valid_low})
        
        if val_loss < best_loss:
            save_path = saver.save(sess, "./mnist_model_ex_8_2")
            best_loss = val_loss
            Early_checker = 0
            
        else:
            Early_checker += 1
            
            if Early_checker > Early_stop:
                print("Early Stop condition satisfied.")
                break
        
        print("epoch: {}\tVal_loss: {:.5f}\tBest_loss: {:.5f}\tVal_Acc: {:.2f} %".
              format(epoch, val_loss, best_loss, 100*val_acc))




epoch: 0	Val_loss: 0.17101	Best_loss: 0.17101	Val_Acc: 96.29 %
epoch: 1	Val_loss: 0.10023	Best_loss: 0.10023	Val_Acc: 97.69 %
epoch: 2	Val_loss: 0.09042	Best_loss: 0.09042	Val_Acc: 97.85 %
epoch: 3	Val_loss: 0.09084	Best_loss: 0.09042	Val_Acc: 97.97 %
epoch: 4	Val_loss: 0.09744	Best_loss: 0.09042	Val_Acc: 98.20 %
epoch: 5	Val_loss: 0.07220	Best_loss: 0.07220	Val_Acc: 98.20 %
epoch: 6	Val_loss: 0.40714	Best_loss: 0.07220	Val_Acc: 95.86 %
epoch: 7	Val_loss: 0.07249	Best_loss: 0.07220	Val_Acc: 98.44 %
epoch: 8	Val_loss: 0.07518	Best_loss: 0.07220	Val_Acc: 97.85 %
epoch: 9	Val_loss: 0.05422	Best_loss: 0.05422	Val_Acc: 98.67 %
epoch: 10	Val_loss: 0.05847	Best_loss: 0.05422	Val_Acc: 98.75 %
epoch: 11	Val_loss: 0.07658	Best_loss: 0.05422	Val_Acc: 98.71 %
epoch: 12	Val_loss: 0.07264	Best_loss: 0.05422	Val_Acc: 98.28 %
epoch: 13	Val_loss: 0.21070	Best_loss: 0.05422	Val_Acc: 96.40 %
epoch: 14	Val_loss: 0.18337	Best_loss: 0.05422	Val_Acc: 98.20 %
epoch: 15	Val_loss: 0.14284	Best_loss: 0.05422	Val

In [10]:
with tf.Session() as sess:
    saver.restore(sess, "./mnist_model_ex_8_2")
    test_acc = accuracy.eval(feed_dict={X: X_test_low, y: y_test_low})
    print("Test Accuracy: {:.2f} %".format(100*test_acc))

INFO:tensorflow:Restoring parameters from ./mnist_model_ex_8_2
Test Accuracy: 98.83 %


### 8.3)
Tune the hyperparameters using cross-validation and see what precision you can achieve.